In [12]:
import os
import cv2
import re
import pytesseract
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments


In [14]:
# Path to cosmetic ingredient images
image_dir = "C:\\Users\\LENOVO\\Desktop\\DSGP data\\only labels - Copy"
cosmetic_texts = []

# Extract text from each image
for img_name in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_name)
    image = cv2.imread(img_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    text = pytesseract.image_to_string(thresh)
    cosmetic_texts.append(text)

In [17]:
import re

# List to store extracted ingredients
extracted_ingredients = []

# Regular expression pattern to match valid ingredient names (only alphabetic characters and hyphens)
ingredient_pattern = r'\b[A-Za-z\-]+\b'

# Set to store unique, valid ingredients
unique_ingredients = set()

# Loop through each text in the list of cosmetic texts
for text in cosmetic_texts:
    # Lowercase the text and find all matches of ingredients using the regex pattern
    ingredients = [ingredient.lower() for ingredient in re.findall(ingredient_pattern, text)]
    # Filter out single-letter, two-letter, and three-letter words
    filtered_ingredients = [ingredient for ingredient in ingredients if len(ingredient) > 3]
    # Add filtered ingredients to the set to ensure uniqueness
    unique_ingredients.update(filtered_ingredients)

# Convert the set back to a sorted list (optional, if order matters)
extracted_ingredients = sorted(unique_ingredients)

# Print the cleaned and extracted ingredients
print(extracted_ingredients)


['-aaay', '-amodimetbicone', '-buteth-', '-ceteth-', '-diaminophenoxyethanol', '-dodecylbenzenesulfonate', '-ethyt', '-glucosipg', '-hydroxybenzoic', '-isethiezelinene', '-isot', '-isothi', '-isothiazolino', '-isothiazolinone', '-isothiazotinone', '-methy', '-methyl-', '-methyr', '-methyt-', '-o-ethy', '-o-ethyi', '-o-ethyl', '-panthenol', '-phenoxyethanol', '-sulfolaurate', '-t-butyl', 'a-ycerin', 'aaannarniua', 'aaganasphosarernel', 'aalee', 'aang', 'aaocorbic', 'aati', 'aaua', 'aber', 'absiol', 'aburm', 'acai', 'acamide', 'acamlates', 'acbl', 'acer', 'aces', 'acetanilid', 'acetat', 'acetate', 'acetic', 'acetum', 'acetyl', 'ache', 'acid', 'acids', 'acinamig', 'acip', 'acitate', 'acoh', 'acone', 'acorn', 'acres', 'acry', 'acrylaces', 'acrylate', 'acrylates', 'acrylic', 'acryloylainety', 'acta', 'actd', 'acted', 'activated', 'actylate', 'acua', 'acvote', 'adenanthera', 'adroiee', 'ae-vlate', 'aegagrus', 'aero', 'aert', 'aeua', 'aeyen', 'afaecum', 'afta', 'afte', 'after', 'agaee', 'agid

In [19]:
import json

# Save the extracted ingredients to a JSON file
with open('extracted_ingredients.json', 'w') as json_file:
    json.dump(extracted_ingredients, json_file, indent=4)

print("Ingredients saved to extracted_ingredients.json")


Ingredients saved to extracted_ingredients.json


In [21]:

# Load the extracted ingredients from the JSON file
with open('extracted_ingredients.json', 'r') as json_file:
    loaded_ingredients = json.load(json_file)

# Print the loaded ingredients
print(loaded_ingredients)


['-aaay', '-amodimetbicone', '-buteth-', '-ceteth-', '-diaminophenoxyethanol', '-dodecylbenzenesulfonate', '-ethyt', '-glucosipg', '-hydroxybenzoic', '-isethiezelinene', '-isot', '-isothi', '-isothiazolino', '-isothiazolinone', '-isothiazotinone', '-methy', '-methyl-', '-methyr', '-methyt-', '-o-ethy', '-o-ethyi', '-o-ethyl', '-panthenol', '-phenoxyethanol', '-sulfolaurate', '-t-butyl', 'a-ycerin', 'aaannarniua', 'aaganasphosarernel', 'aalee', 'aang', 'aaocorbic', 'aati', 'aaua', 'aber', 'absiol', 'aburm', 'acai', 'acamide', 'acamlates', 'acbl', 'acer', 'aces', 'acetanilid', 'acetat', 'acetate', 'acetic', 'acetum', 'acetyl', 'ache', 'acid', 'acids', 'acinamig', 'acip', 'acitate', 'acoh', 'acone', 'acorn', 'acres', 'acry', 'acrylaces', 'acrylate', 'acrylates', 'acrylic', 'acryloylainety', 'acta', 'actd', 'acted', 'activated', 'actylate', 'acua', 'acvote', 'adenanthera', 'adroiee', 'ae-vlate', 'aegagrus', 'aero', 'aert', 'aeua', 'aeyen', 'afaecum', 'afta', 'afte', 'after', 'agaee', 'agid

In [25]:
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input

ModuleNotFoundError: No module named 'tensorflow'

In [23]:
vectorizer = TfidfVectorizer(max_features=5000)
X_vec = vectorizer.fit_transform(loaded_ingredients).toarray()

NameError: name 'TfidfVectorizer' is not defined

In [27]:

# Split into train and test
X_train, X_test = train_test_split(X_vec, test_size=0.2, random_state=42)

NameError: name 'X_vec' is not defined

In [ ]:
# Define the autoencoder model
input_dim = X_train.shape[1]
autoencoder = Sequential([
    Dense(256, activation='relu', input_dim=input_dim),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(input_dim, activation='sigmoid')
])

In [ ]:
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train, X_train, epochs=20, batch_size=32, validation_data=(X_test, X_test))


In [ ]:
import re
import pytesseract
from PIL import Image

# Function to extract text from an image using OCR
def extract_text_from_image(image_path):
    # Use pytesseract to perform OCR on the image
    return pytesseract.image_to_string(Image.open(image_path))

# Function to clean and preprocess the extracted text
def preprocess_ingredients(text):
    # Split the text by commas
    raw_ingredients = text.split(',')
    # Regular expression pattern to match valid ingredient names (only alphabetic characters and hyphens)
    ingredient_pattern = r'\b[A-Za-z\-]+\b'
    cleaned_ingredients = []
    
    for ingredient in raw_ingredients:
        # Remove special characters and lowercase each ingredient
        matches = [match.lower() for match in re.findall(ingredient_pattern, ingredient)]
        # Filter out single-letter, two-letter, and three-letter words
        filtered = [match for match in matches if len(match) > 3]
        # Add cleaned ingredients to the list
        cleaned_ingredients.extend(filtered)

    # Return the cleaned and unique ingredients as a list
    return sorted(set(cleaned_ingredients))

# Input: Path to the image containing the ingredient list
image_path = "Untitled6.jpg"  # Replace with the actual image path

# Step 1: Extract text from the image
extracted_text = extract_text_from_image(image_path)

# Step 2: Preprocess the extracted text to get the ingredient list
ingredient_list = preprocess_ingredients(extracted_text)

# Print the final ingredient list
print("Extracted Ingredient List:", ingredient_list)

# Example of using this list for further evaluation
# Assuming you already have the vectorizer and autoencoder set up
new_data_vec = vectorizer.transform([" ".join(ingredient_list)]).toarray()
reconstructions = autoencoder.predict(new_data_vec)


In [ ]:
# Compute reconstruction error
threshold = np.mean(np.sum((X_train - autoencoder.predict(X_train))**2, axis=1))
for text, reconstruction in zip(new_data, reconstructions):
    error = np.sum((new_data_vec[0] - reconstruction)**2)
    category = "Cosmetic" if error < threshold else "Non-Cosmetic"
    print(f"'{text}' -> {category} (Error: {error:.4f})")